In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from utils import *
import matplotlib.pyplot as plt
import matplotlib

data_path = '../data/FACT_raw.xlsx'

df_all = pd.read_excel(
    data_path, engine='openpyxl', sheet_name=None)['Sheet1']

df_all.drop(0,axis=0,inplace=True)
df_all.reset_index(drop=True,inplace=True)

In [2]:
name_mapping = {
    # 'optiDAT nr.', ##### Not useful
    # 'Optimat/FACT name', ##### Not useful
    # 'data delivered under name', ##### Not useful
    # 'from plate',##### empty
    # 'tested at laboratory', ##### Not useful
    'laminate': 'Lay-up',
    # 'cut angle': 'Cut Angle', ##### empty
    # 'chop', ##### What is it?
    0: 'Percentage of Fibre in 0-deg Direction',
    '+/-45': 'Percentage of Fibre in 45-deg Direction',
    90: 'Percentage of Fibre in 90-deg Direction',
    'other direction': 'Percentage of Fibre in Other Direction',
    # 'Resin', ##### Not useful
    # 'Fibre', ##### Not useful
    'FVF': 'Fibre Volumn Fraction',
    'porosity': 'Porosity', ##### Too much absence
    'Barcol hardness': 'Barcol Hardness',
    # 'Manufactured by/Quality Assurance Specification', ##### Not useful
    # 'production method',##### Not useful
    # 'Geometry', ##### Not useful
    # 'material ', ##### Not useful
    'thickness (old)': 'Thickness',
    # 'average thickness (new)', ##### empty
    'maximum width': 'Maximum Width',
    'minimum width': 'Minimum Width',
    # 'average width (new)', ##### empty
    'area': 'Area',
    'length': 'Length',
    'load length': 'Load Length',
    'radius of waist': 'Radius of Waist',
    # 'specimen standard', ##### empty
    # 'prog', ##### Not useful
    # 'start test', ##### Not useful
    # 'end of test', ##### Not useful
    # 'test type', ##### As filter
    'R-value': 'Minimum/Maximum Stress',
    # 'F_max',##### empty
    # 'Fmax during fatigue', ##### empty
    # 'Fmax during static test', ##### empty
    # 'deviation of Fmax>2% w.r.t. test frame settings during fatigue?', ##### empty
    # 'F_max 90° direction', ##### empty
    'e_max': 'Maximum Strain',
    # 'e_max 90°', ##### empty
    # 'e_max 45°', ##### empty
    's_max': 'Maximum Stress',
    # 'normalized s_max', ##### Not useful
    # 'shear strain (e12)', ##### empty
    # 'shear strength', ##### empty
    # 'Shear Modulus (G12)', ##### empty
    'No. of cycles to failure': 'Cycles to Failure',
    # 'level', ##### empty
    # 'Unnamed: 51', ##### empty
    # 'no. of cycles', ##### empty
    # 'Unnamed: 53', ##### empty
    # 'No. of spectrum passes to failure', ##### To much absence, how to use it?
    # 'failure mode', ##### To much absence
    # 'runout?',
    # 'R-value.1', ##### empty
    # 'F_max.1', ##### empty
    # 'e_max.1', ##### empty
    # 's_max.1', ##### empty
    # 'No. of cycles in block 2', ##### empty
    'UTS': 'Static Maximum Tensile Stress',
    'UCS': 'Static Maximum Compressive Stress',
    'e_UTS': 'Static Maximum Tensile Strain',
    'e_UCS': 'Static Maximum Compressive Strain',
    # 'UTS (mat. spec.)', ##### What is it?
    # 'UCS (mat. spec.)', ##### What is it?
    # 'corr. UTS', ##### What is it?
    # 'corr. UCS', ##### What is it?
    # 'RTS', ##### empty
    # 'RCS', ##### empty
    # 'e_RTS', ##### empty
    # 'e_RCS', ##### empty
    # 'LRU': 'Stress Rate',
    'SRU': 'Strain Rate',
    # 'SRU.1', ##### empty
    # 'SRU speed deviation ? 2% during static test?', ##### empty
    # 'LRF', ##### What is it?
    # 'SRF', ##### What is it?
    # 'SRF.1', ##### What is it?
    # 'wave', ##### Not useful
    'fconstant': 'Frequency',
    # 'test time', ##### Not useful
    # 'fconstant block 2', ##### empty
    'Eit': 'Static Elastic Modulus',
    'Eic': 'Static Compressive Modulus',
    # 'Eft',
    # 'Efc',
    # 'ILSS', ##### empty
    # 's_flex', ##### empty
    # 'test machine', ##### empty
    # 'control', ##### All load control
    # 'grip', ##### Not useful
    # 'ABG', ##### What is it?
    'Temperature': 'Temperature',
    # 'Temperature control?',
    # 'Preconditioned?', ##### empty
    # 'environment', ##### All "d", What is it?
    'RH': 'Relative Humidity',
    # 'Test condition', ##### empty
    # 'ref.', ##### Not useful
    # 'NB', ##### What is it?
    # 'RB', ##### What is it?
    # 's_res', ##### empty
    # 'N10sr', ##### To much absence
    # 'specimen shape', ##### Not useful
    # 'Remarks', ##### empty
    # 'Bending', ##### empty
    # 'buckling', ##### empty
    # 'temperature failure or temperature above 35 °C', ##### empty
    # 'tab failure', ##### empty
    # 'delaminated', ##### empty
    # 'incomplete measurement data available',  ##### empty
    # 'Strain calculated using E', ##### empty
    # 'Premature failure in RST', ##### empty
    # "Poissons's ratio", ##### empty
    # 'Strain measurement equipment', ##### empty
    # 'Strain measurement equipment (2)', ##### empty
    # 'Grip pressure' ##### empty
}

In [3]:
plot_absence(df_all, name_mapping, '../data/FACT_absence_ratio_initial.png', fontsize=12)

In [4]:
modify_col(df_all, 0, fill_na, n = 0)
modify_col(df_all, '+/-45', fill_na, n = 0)
modify_col(df_all, 90, fill_na, n = 0)
modify_col(df_all, 'other direction', fill_na, n = 0)

In [5]:
# Add material code
material_code = [str(df_all.loc[x, 'material ']) + str(df_all.loc[x, 'laminate']) for x in range(len(df_all))]
df_all['Material_Code'] = material_code

In [6]:
# If R>1, max stress must be negative
where_R1 = []
for idx,x in enumerate(df_all['R-value']):
    if not isinstance(x,str) and x>1:
        where_R1.append(idx)
where_R1 = np.array(where_R1)
df_all.loc[where_R1,'s_max'] = -np.abs(df_all.loc[where_R1,'s_max'])

In [7]:
# Cal minimum stress
df_all['Minimum Stress'] = df_all['R-value'] * df_all['s_max']

In [8]:
if 'laminate' in df_all.columns:
    code2seq_dict = {}
    layups = df_all['laminate'].values
    for layer in list(set(layups)):
        code2seq_dict[layer] = code2seq(layer)
    seq = []
    for layer in layups:
        seq.append('/'.join([str(x) for x in code2seq(layer)]))
    df_all['Sequence'] = seq

In [9]:
df_fatigue = df_all.loc[np.where(df_all['test type'] == 'CA')[0],:].copy()
df_fatigue.reset_index(drop=True, inplace=True)
df_fatigue = replace_column_name(df_fatigue, name_mapping)

In [10]:
# Remove runout tests
df_fatigue = df_fatigue.loc[np.where(df_fatigue['runout?'] != 'y')[0],:].copy()
df_fatigue.reset_index(drop=True, inplace=True)

In [11]:
df_fatigue['log(Static Maximum Tensile Stress)'] = np.log10(df_fatigue['Static Maximum Tensile Stress'].values.astype(float))
df_fatigue['log(Cycles to Failure)'] = np.log10(df_fatigue['Cycles to Failure'].values.astype(float))

In [12]:
df_fatigue.to_excel('../data/FACT_fatigue.xlsx', engine='openpyxl', index=False)

In [13]:
averaging(df_fatigue, measure_features=['Maximum Stress', 'Minimum Stress', 'Frequency']).to_excel('../data/FACT_avg_fatigue.xlsx', engine='openpyxl', index=False)

  0%|          | 0/34 [00:00<?, ?it/s]

/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [14]:
averaging(df_fatigue.copy(), measure_features=['log(Static Maximum Tensile Stress)']).to_excel('../data/FACT_avg_static.xlsx', engine='openpyxl', index=False)

  0%|          | 0/34 [00:00<?, ?it/s]

/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: Ru

In [15]:
plot_absence(df_fatigue, name_mapping, '../data/FACT_absence_ratio.png', fontsize=5)